# pandas_groupby常用8个用法

In [1]:
import pandas as pd
from sklearn import datasets

# 加载数据集和目标
data, target = datasets.load_iris(return_X_y=True, as_frame=True)
# 合并数据集和目标
iris = pd.concat([data, target], axis=1, sort=False)
iris

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [2]:
iris.sample(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
108,6.7,2.5,5.8,1.8,2
35,5.0,3.2,1.2,0.2,0
83,6.0,2.7,5.1,1.6,1
96,5.7,2.9,4.2,1.3,1
76,6.8,2.8,4.8,1.4,1


In [3]:
iris_gb = iris.groupby('target')
type(iris_gb)

pandas.core.groupby.generic.DataFrameGroupBy

## 创建频率表
假如我想知道每个species类中的数量有多少，那么直接使用groupby的size函数即可

In [4]:
iris_gb.size()

target
0    50
1    50
2    50
dtype: int64

## 计算常用的描述统计量

min、max()、medianhe、std等

In [5]:
# 计算均值
iris_gb.mean()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
target,,,,
0,5.006,3.428,1.462,0.246
1,5.936,2.770,4.260,1.326
2,6.588,2.974,5.552,2.026


In [6]:
# 单列
iris_gb['sepal length (cm)'].mean()

target
0    5.006
1    5.936
2    6.588
Name: sepal length (cm), dtype: float64

In [7]:
# 双列
iris_gb[['sepal length (cm)', 'sepal width (cm)']].mean()

,sepal length (cm),sepal width (cm)
target,,
0,5.006,3.428
1,5.936,2.770
2,6.588,2.974


## 查找最大值（最小值）索引
查找每个组的最大值或最小值的索引

In [8]:
iris_gb.idxmax()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
target,,,,
0,14,15,24,43
1,50,85,83,70
2,131,117,118,100


想查找每组sepal_length最大值对应的整条记录时，就可以这样用。注意，这里是整条记录，相当于按sepal_length最大值这个条件进行了筛选。

In [9]:
sepal_largest = iris.loc[iris_gb['sepal length (cm)'].idxmax()]
sepal_largest

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
14,5.8,4.0,1.2,0.2,0
50,7.0,3.2,4.7,1.4,1
131,7.9,3.8,6.4,2.0,2


## Groupby之后重置索引
很多时候，在groupby处理后还要进行其他操作。也就是说，我们想重置分组索引以使其成为正常的行和列。
第一种方法可能大家常用，就是通过reset_index()让乱序索引重置。

In [10]:
iris_gb.max().reset_index()

,target,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0,5.8,4.4,1.9,0.6
1,1,7.0,3.4,5.1,1.8
2,2,7.9,3.8,6.9,2.5


但其实，还有一个看上去更加友好的用法。可以在groupby的时候就设置as_index参数，也可以达到同样效果。

In [11]:
iris.groupby('target', as_index=False).max()

,target,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0,5.8,4.4,1.9,0.6
1,1,7.0,3.4,5.1,1.8
2,2,7.9,3.8,6.9,2.5


## 多种统计量汇总
上面都是单个统计量的操作，那如果我想同时操作好几个呢？

groupby还有一个超级棒的用法就是和聚合函数agg连起来使用。

In [12]:
iris_gb[['sepal length (cm)', 'sepal width (cm)']].agg(["min", "mean"])

sepal length (cm)        sepal width (cm)       
                     min   mean              min   mean
target                                                 
0                    4.3  5.006              2.3  3.428
1                    4.9  5.936              2.0  2.770
2                    4.9  6.588              2.2  2.974

## 特定列的聚合
我们也看到了，上面是的多个操作对于每个列都是一样的。实际使用过程中，我们可能对于每个列的需求都是不一样的。

所以在这种情况下，我们可以通过为不同的列单独设置不同的统计量。

In [13]:
iris_gb.agg({"sepal length (cm)": ["min", "max"], "sepal width (cm)": ["mean", "std"]})

sepal length (cm)      sepal width (cm)          
                     min  max             mean       std
target                                                  
0                    4.3  5.8            3.428  0.379064
1                    4.9  7.0            2.770  0.313798
2                    4.9  7.9            2.974  0.322497

## NamedAgg命名统计量
现在我又有新的想法了。上面的多级索引看起来有点不太友好，我想把每个列下面的统计量和列名分别合并起来。可以使用NamedAgg来完成列的命名。

In [14]:
iris_gb.agg(
     sepal_min=pd.NamedAgg(column="sepal length (cm)", aggfunc="min"),
     sepal_max=pd.NamedAgg(column="sepal length (cm)", aggfunc="max"),
     petal_mean=pd.NamedAgg(column="petal length (cm)", aggfunc="mean"),
     petal_std=pd.NamedAgg(column="petal length (cm)", aggfunc="std")
 )

,sepal_min,sepal_max,petal_mean,petal_std
target,,,,
0,4.3,5.8,1.462,0.173664
1,4.9,7.0,4.260,0.469911
2,4.9,7.9,5.552,0.551895


因为NamedAgg是一个元组，所以我们也可以直接赋值元组给新的命名，效果一样，但看上去更简洁。

In [15]:
iris_gb.agg(
    sepal_min=("sepal length (cm)", "min"),
    sepal_max=("sepal length (cm)", "max"),
    petal_mean=("petal length (cm)", "mean"),
    petal_std=("petal length (cm)", "std")
)

,sepal_min,sepal_max,petal_mean,petal_std
target,,,,
0,4.3,5.8,1.462,0.173664
1,4.9,7.0,4.260,0.469911
2,4.9,7.9,5.552,0.551895


## 使用自定义函数
上面agg聚合函数中我们都是通过添加一个统计量名称来完成操作的，除此之外我们也可直接给一个功能对象。

In [16]:
iris_gb.agg(pd.Series.mean)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
target,,,,
0,5.006,3.428,1.462,0.246
1,5.936,2.770,4.260,1.326
2,6.588,2.974,5.552,2.026


In [17]:
# 不仅如此，名称和功能对象也可一起使用。
iris_gb.agg(["min", pd.Series.mean])

sepal length (cm)        sepal width (cm)        petal length (cm)  \
                     min   mean              min   mean               min   
target                                                                      
0                    4.3  5.006              2.3  3.428               1.0   
1                    4.9  5.936              2.0  2.770               3.0   
2                    4.9  6.588              2.2  2.974               4.5   

              petal width (cm)         
         mean              min   mean  
target                                 
0       1.462              0.1  0.246  
1       4.260              1.0  1.326  
2       5.552              1.4  2.026

In [18]:
# 我们还可以自定义函数，也都是可以的。
def double_length(x):
    return 2*x.mean()

iris_gb.agg(double_length)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
target,,,,
0,10.012,6.856,2.924,0.492
1,11.872,5.540,8.520,2.652
2,13.176,5.948,11.104,4.052


In [19]:
# 如果想更简洁，也可以使用lambda函数。总之，用法非常灵活，可以自由组合搭配。
iris_gb.agg(lambda x: x.mean())

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
target,,,,
0,5.006,3.428,1.462,0.246
1,5.936,2.770,4.260,1.326
2,6.588,2.974,5.552,2.026
